Assignment 1 <br>
Student name: Tim Kleinlein <br>
Matriculation Nr: 01/1151277
Other group members: 1151248, 1152810,  919755

1)

In [61]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [76]:
def zmaker(Y,p,intercept):
    """
    create matrix Z when given time series data Y, lag order p and boolean whether to include intercept in estimation.
    """
    k = Y.shape[0]  # number of variables
    if intercept:
        Z = np.zeros(shape=(k*p+1,Y.shape[1]-p)) # empty Z placeholder array which will be filled with correct values later
        Z[0] = 1  # intercept
    else:
        Z = np.zeros(shape=(k*p,Y.shape[1]-p))  # empty Z placeholder array which will be filled with correct values later
    
    for i in range(p,Y.shape[1]): # individual z vectors which will be inserted in Z
        z = np.array([]) # placeholder array for z
        for x in range(1, p+1):  # for all lags p
            for j in range(k): # and all variables k
                z = np.append(z,Y[j, i-x])  # get correct y values and create z
        if intercept:
            Z[1:,i-p] = z  # if intercept is estimated first row is already filled with 1's
        else:
            Z[:,i-p] = z
    return Z
            

In [169]:
def estimator(Y, p, intercept):
    """
    estimate VAR(p) for given time series data Y, lag order p and boolean whether to include intercept in estimation. 
    Return estimated coefficients, their t-ratios and the covariance matrix.
    """
    Z = zmaker(Y,p, intercept)  # create matrix Z
    Y = Y[:,p:]  # first p observations are lost as we need prior lags for estimation
    B = Y @ Z.transpose() @ inv((Z@Z.transpose()))  # calculate estimated coefficients
    
    T = Y.shape[1]  # number observations used for estimation
    K = Y.shape[0]  # number of variables
    cov_matrix = (1/(T-K*p-1))*((Y-(B@Z))@(Y-(B@Z)).transpose())  # calculate covariance matrix
    
    t_matrix = np.kron(inv(Z@Z.transpose()),cov_matrix)  # calculate matrix T to get coefficients variances(diagonals)
    std_errors = np.sqrt(np.diag(t_matrix))  # standard errors used for t-ratios are squared variance
    t_values=[]  #. placeholder list for t values
    std_error_counter=0
    for j in range(B.shape[1]):  # choose this loop structure to assure that coefficient is divided by its according standard error
        for i in range(K):
            t_values.append(B[i,j]/std_errors[std_error_counter])
            std_error_counter+=1
    t_matrix = np.array(t_values) # convert list to array
    t_matrix = t_matrix.reshape(B.shape[1],K).transpose()  # reshape array such that is same format as coefficients: K x (1+K*p)
    
    return B, t_matrix, cov_matrix

2)

In [62]:
fed = pd.read_csv('FEDFUNDS.csv', delimiter=',')
gdp = pd.read_csv('GDPC1.csv', delimiter=',')
gdp_def = pd.read_csv('GDPDEF.csv', delimiter=',')

In [63]:
# create np arrays
gdp = np.array(gdp['GDPC1'])
gdp_def = np.array(gdp_def['GDPDEF'])
fed =  np.array(fed['FEDFUNDS'][:-1])

In [64]:
# convert values to floats
floatmaker = lambda x: float(x)
floatmaker2 = np.vectorize(floatmaker)
gdp = floatmaker2(gdp)
gdp_def = floatmaker2(gdp_def)
fed = floatmaker2(fed)

In [65]:
#create variables for estimation
gdp = 100*(np.ediff1d(gdp))
gdp_def = 100*(np.ediff1d(gdp_def))
fed = fed[1:]  # due to difference first observation lost

In [66]:
# combine time series in matrix to use as input for function
Y = np.array([gdp, gdp_def, fed])
Y.shape

(3, 250)

In [178]:
B, t_ratios, cov_matrix = estimator(Y,4,True)

In [179]:
B

array([[ 1.29215143e+04, -1.81269795e-01, -4.14861144e+01,
         1.26480458e+03, -8.06846277e-02,  1.00877612e+02,
        -1.69122182e+03, -7.76354625e-02,  4.43516543e+01,
        -1.45431228e+02, -4.48135541e-02, -1.13627704e+02,
        -8.48213330e+01],
       [ 7.12236033e+00, -4.32297228e-05,  5.55208972e-01,
         1.67915553e+00,  1.68156250e-04,  1.83466157e-01,
        -9.96156297e-01, -1.24780708e-04,  2.26247098e-01,
        -7.91991464e-01, -1.07615633e-04, -2.71749031e-02,
        -4.06935239e-01],
       [ 1.57070276e-01,  6.35263343e-06,  2.31084267e-04,
         8.79792900e-01, -1.38907291e-06,  8.33584830e-03,
        -5.90343977e-02,  7.42639560e-07, -4.71539194e-03,
         3.49594543e-01,  8.78753972e-07, -4.26613052e-03,
        -2.10160578e-01]])

In [180]:
t_ratios

array([[ 5.49428062, -2.4582855 , -0.67508766,  1.43364424, -1.06901706,
         1.36251069, -1.48512015, -1.01527079,  0.58946513, -0.12742552,
        -0.65458737, -1.66444861, -0.09364458],
       [ 2.29675217, -0.44461294,  6.85183116,  1.44344972,  1.68966115,
         1.87929024, -0.66340887, -1.23754774,  2.28046851, -0.52627413,
        -1.19213756, -0.30188875, -0.34071844],
       [ 0.91116704,  1.17535008,  0.05130201, 13.60520184, -0.25108771,
         1.53603637, -0.70724963,  0.13249711, -0.85501289,  4.17897685,
         0.17511853, -0.85256377, -3.16544952]])

In [181]:
cov_matrix

array([[2.35823048e+08, 1.31647245e+05, 2.21863173e+03],
       [1.31647245e+05, 4.10015909e+02, 1.24478162e+00],
       [2.21863173e+03, 1.24478162e+00, 1.26698976e+00]])

For personal checking only, not relevant for assignment solution!

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [182]:
model = VAR(Y.transpose())

In [183]:
results = model.fit(4)

In [184]:
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 09, Nov, 2022
Time:                     16:43:26
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    26.0272
Nobs:                     246.000    HQIC:                   25.6953
Log likelihood:          -4141.17    FPE:                1.15413e+11
AIC:                      25.4715    Det(Omega_mle):     9.88921e+10
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const     12921.514307      2351.811858            5.494           0.000
L1.y1        -0.181270         0.073738           -2.458           0.014
L1.y2       -41.486114        61.452930           -0.675           0.500
L1.y3      1264.804577